In [3]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
data_root = "/mnt/data/geolifeclef-2024/data"

survey_stats = spark.read.parquet(
    f"{data_root}/processed/geolsh_nn_graph/v2/survey_stats"
)
species_stats = spark.read.parquet(
    f"{data_root}/processed/geolsh_nn_graph/v2/species_stats"
)
timing = spark.read.json(f"{data_root}/processed/geolsh_nn_graph/v2/timing")
survey_stats.printSchema()
species_stats.printSchema()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


24/04/07 09:09:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- summary: string (nullable = true)
 |-- n: string (nullable = true)
 |-- count: string (nullable = true)
 |-- threshold: integer (nullable = true)
 |-- name: string (nullable = true)

root
 |-- summary: string (nullable = true)
 |-- n: string (nullable = true)
 |-- count: string (nullable = true)
 |-- threshold: integer (nullable = true)
 |-- name: string (nullable = true)



In [11]:
timing.orderBy("threshold").show()

+----------+------------------+------------------+---------+
|  df_count|      time_species|       time_survey|threshold|
+----------+------------------+------------------+---------+
| 398280064|120.23328977100027|     172.096345252|    10000|
| 524694680|130.59163260599962|205.71591510600047|    20000|
| 633669808|130.19804506199944|225.08103482300066|    30000|
| 724795718| 133.9478949229997|236.56537008600026|    40000|
| 806658192|135.57080295499873|261.90445328199894|    50000|
| 879183638|142.68195944999934| 277.4817794949995|    60000|
| 949433228|148.49451990600028|  301.789006695999|    70000|
|1010857892|150.32977327299886| 336.8670750399997|    80000|
|1066338158|164.46621430099913| 365.2691382770008|    90000|
|1116195844|167.60283644500123| 372.0538387280012|   100000|
+----------+------------------+------------------+---------+



In [6]:
survey_stats.show()

+-------+------------------+-----+---------+----+
|summary|                 n|count|threshold|name|
+-------+------------------+-----+---------+----+
|  count|         213169116| NULL|    80000|freq|
|   mean| 4.742046648070727| NULL|    80000|freq|
| stddev| 16.65080311131445| NULL|    80000|freq|
|    min|                 1| NULL|    80000|freq|
|    max|             14487| NULL|    80000|freq|
|  count|         225775632| NULL|    90000|freq|
|   mean| 4.722999327048722| NULL|    90000|freq|
| stddev|16.668448913836965| NULL|    90000|freq|
|    min|                 1| NULL|    90000|freq|
|    max|             14487| NULL|    90000|freq|
|  count|         237508548| NULL|   100000|freq|
|   mean| 4.699602828610615| NULL|   100000|freq|
| stddev| 16.66624476198729| NULL|   100000|freq|
|    min|                 1| NULL|   100000|freq|
|    max|             14487| NULL|   100000|freq|
|  count|         152228205| NULL|    40000|freq|
|   mean| 4.761244593273632| NULL|    40000|freq|


In [8]:
print("survey to species network")
survey_stats.where(F.col("name") == "freq").groupBy("threshold").pivot("summary").agg(
    F.first(F.round("n", 2))
).orderBy("threshold").show()

survey_stats.where(F.col("name") == "degree").groupBy("threshold").pivot("summary").agg(
    F.first(F.round("count", 2))
).orderBy("threshold").show()

survey to species network


+---------+------------+-------+----+---+------+
|threshold|       count|    max|mean|min|stddev|
+---------+------------+-------+----+---+------+
|    10000| 7.7767162E7| 4048.0|5.12|1.0| 17.92|
|    20000|1.08093668E8| 4048.0|4.85|1.0| 16.84|
|    30000|1.32068825E8| 4048.0| 4.8|1.0| 16.56|
|    40000|1.52228205E8|14487.0|4.76|1.0| 16.51|
|    50000|1.69809051E8|14487.0|4.75|1.0| 16.53|
|    60000|1.85561009E8|14487.0|4.74|1.0| 16.51|
|    70000|1.99894112E8|14487.0|4.75|1.0| 16.59|
|    80000|2.13169116E8|14487.0|4.74|1.0| 16.65|
|    90000|2.25775632E8|14487.0|4.72|1.0| 16.67|
|   100000|2.37508548E8|14487.0| 4.7|1.0| 16.67|
+---------+------------+-------+----+---+------+



+---------+------+------------+---------+---+----------+
|threshold| count|         max|     mean|min|    stddev|
+---------+------+------------+---------+---+----------+
|    10000|1201.0| 5.1081767E7| 64752.01|1.0|1503929.99|
|    20000|1207.0| 7.1192966E7| 89555.65|1.0|2093658.89|
|    30000|1214.0| 8.6680251E7|108788.16|1.0|2544528.03|
|    40000|1233.0| 9.9696313E7|123461.64|1.0|2906190.41|
|    50000|1232.0|1.11168157E8|137832.02|1.0|3242995.22|
|    60000|1242.0|1.21611065E8| 149405.0|1.0|3533541.26|
|    70000|1251.0|1.31179239E8|159787.46|1.0|3797525.86|
|    80000|1256.0|1.40232022E8|169720.63|1.0|4050947.91|
|    90000|1260.0|1.48898492E8|179187.01|1.0|4294002.45|
|   100000|1270.0|1.57054862E8| 187014.6|1.0|4510726.58|
+---------+------+------------+---------+---+----------+



In [9]:
print("species to species network")
species_stats.where(F.col("name") == "freq").groupBy("threshold").pivot("summary").agg(
    F.first(F.round("n", 2))
).orderBy("threshold").show()

species_stats.where(F.col("name") == "degree").groupBy("threshold").pivot(
    "summary"
).agg(F.first(F.round("count", 2))).orderBy("threshold").show()

species to species network


+---------+-----------+---------+------+---+-------+
|threshold|      count|      max|  mean|min| stddev|
+---------+-----------+---------+------+---+-------+
|    10000|  7233877.0|1317572.0| 55.06|1.0| 903.63|
|    20000|  8150955.0|1319838.0| 64.37|1.0|  921.3|
|    30000|  8763109.0|1319992.0| 72.31|1.0| 957.06|
|    40000|  9237555.0|1320068.0| 78.46|1.0| 991.42|
|    50000|  9637613.0|1320090.0|  83.7|1.0|1026.23|
|    60000|  9988433.0|1320218.0| 88.02|1.0|1056.33|
|    70000|1.0293297E7|1320224.0| 92.24|1.0|1089.46|
|    80000|1.0568075E7|1320226.0| 95.65|1.0|1117.19|
|    90000|1.0818335E7|1320348.0| 98.57|1.0| 1139.8|
|   100000|1.1039345E7|1320352.0|101.11|1.0|1159.08|
+---------+-----------+---------+------+---+-------+



+---------+-------+---------+------+---+--------+
|threshold|  count|      max|  mean|min|  stddev|
+---------+-------+---------+------+---+--------+
|    10000| 8958.0|2081776.0|807.53|1.0|25891.29|
|    20000|10413.0|2292186.0|782.77|1.0|26503.04|
|    30000|11596.0|2426268.0| 755.7|1.0|26627.09|
|    40000|12492.0|2526432.0|739.48|1.0| 26762.2|
|    50000|13245.0|2611442.0|727.64|1.0|26892.21|
|    60000|13900.0|2683700.0|718.59|1.0|27007.11|
|    70000|14571.0|2747612.0|706.42|1.0|27027.77|
|    80000|15065.0|2807240.0| 701.5|1.0|27168.71|
|    90000|15472.0|2858832.0|699.22|1.0|27317.65|
|   100000|15927.0|2905018.0|693.12|1.0|27369.31|
+---------+-------+---------+------+---+--------+

